In [34]:
from os import listdir
from os.path import isfile
from json import load

In [35]:
pasta = "requisicoes"    

In [36]:
total_respondidas = 0
total_sem_responder  = 0
for item in listdir(pasta):
    caminho = f'{pasta}/{item}'
    caminho_resposta = f'{caminho}/resposta.json'
    if isfile(caminho_resposta):
        total_respondidas+=1
    if not isfile(caminho_resposta):
        print("sem responder em: ", item)
        total_sem_responder+=1


print("total: ", total_respondidas + total_sem_responder)
print("total respondidas: ",total_respondidas)
print("total sem responder",total_sem_responder)


total:  26855
total respondidas:  26855
total sem responder 0


In [37]:
clientes = {
    "1":{
        "saldo":0,
        "limite":100000,
    },
    "2":{
        "saldo":0,
        "limite":80000
    },
    "3":{
        "saldo":0,
        "limite":1000000
    },
    "4":{
        "saldo":0,
        "limite":10000000
    },
    "5":{
        "saldo":0,
        "limite":500000
    }
}
ids = clientes.keys()

In [38]:
listagem = listdir(pasta)

for item in listagem:
    caminho = f'{pasta}/{item}'
    caminho_request = f'{caminho}/request.json'
    with open(caminho_request,'r') as arq:
        request = load(arq)
    
    caminho_resposta = f'{caminho}/resposta.json'
    if not isfile(caminho_resposta):
        continue
    
    try:
        rota:str = request['rota']

        rota_dividida = rota.split('/')
        id_cliente = rota_dividida[2]
        if id_cliente not in ids:
            continue
        cliente_atual = clientes[id_cliente]
        
        tipo_rota = rota_dividida[3]
        if tipo_rota != 'transacoes':
            continue

        json_body = request['json_body']
        if json_body['valor'].__class__ == float:
            continue

        if json_body['valor'] < 0:
            raise  Exception("invalido")

        if json_body['valor'].__class__ != int:
            raise  Exception("invalido")

        if  not json_body.get('descricao'):
            continue

        if len(json_body['descricao']) == 0 or len(json_body['descricao'])> 10:
                    continue


        if json_body['tipo'] == 'c':
            cliente_atual['saldo'] += json_body['valor']

        
        if json_body['tipo'] == 'd':
            saldo_e_limite = cliente_atual['saldo'] + cliente_atual['limite']
            if saldo_e_limite - json_body['valor'] < 0:
                #print ('invalida : ',item)
                continue
            cliente_atual['saldo'] -= json_body['valor']

        caminho_resposta = f'{caminho}/resposta.json'
        with open(caminho_resposta,'r') as arq:
            resposta = load(arq)
    except Exception as e:
        print('item', item)
        raise e

In [39]:
clientes

{'1': {'saldo': -72989.2, 'limite': 100000},
 '2': {'saldo': -42998.2, 'limite': 80000},
 '3': {'saldo': -986561.2, 'limite': 1000000},
 '4': {'saldo': -9989752.2, 'limite': 10000000},
 '5': {'saldo': -494828.2, 'limite': 500000}}